## Import Packages

In [2]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

## Read in an Image

In [ ]:
#reading in an image
image = mpimg.imread('solidYellowLeft.jpg')

#printing out some stats and plotting
print('This image is:', type(image), 'with dimensions:', image.shape)
plt.imshow(image)  # if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')

## Helper Functions

In [4]:
import math

def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.

    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)

    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255

    #filling pixels inside the polygon defined by "vertices" with the fill color
    cv2.fillPoly(mask, vertices, ignore_mask_color)

    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=10):
    """
    This function draws `lines` with `color` and `thickness`.
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.

    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img, lines

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.

    `initial_img` should be the image before any processing.

    The result image is computed as follows:

    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)

In [3]:
import os
#os.listdir("test_images/")

## Build a Lane Finding Pipeline



**Reading Image**

In [5]:
from google.colab.patches import cv2_imshow

def read_image(img):

  #image to hsv
  img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

  #useful info
  img_height, img_width = img.shape[:2]

  return img_hsv, img_width, img_height

**Colour Thresholding**

In [6]:
def color_threshold(img_hsv):

  #mask of white
  mask1 = cv2.inRange(img_hsv, (0,0,230), (255,255,255))
  #mask1 = cv2.inRange(img_hsv, (0,0,200), (255,30,255))
  #cv2.imwrite("isthisworking_white.jpg", mask1)

  #debug
  #cv2.imwrite("whitefilter2.jpg", mask1)

  #mask of yellow
  mask2 = cv2.inRange(img_hsv, (15,50,50), (35, 255, 255))
  #mask2 = cv2.inRange(img_hsv, (20,100,100), (40, 255, 255))

  #debug
  #cv2.imwrite("isthisworking_y.jpg", mask2)

  #final mask (white OR yellow)
  img_gray = cv2.bitwise_or(mask1, mask2)

  #debug
  #cv2.imwrite("colorthresh.jpg", img_gray)

  return img_gray

#could tune this by making a combined binary, threshholding in different color schemes
#e.g. https://tjosh.medium.com/finding-lane-lines-with-colour-thresholds-beb542e0d839
#set range of values for hue, allow you to catch bigger range of color in different lighting conditions

**Smoothing**

In [7]:
def smoothing(img_gray):

  img_blur = gaussian_blur(img_gray, 5)

  #debug
  #cv2.imwrite("blur.jpg", img_blur)

  return img_blur

**Canny Edge Detector**

In [8]:
def edge_detector(img_blur):

  img_edges = canny(img_blur, 50, 150)

  #debug
  #cv2.imwrite("canny_50_150.jpg", img_edges)

  return img_edges

**ROI Selection**

In [9]:
def select_roi(img_edges):

  vertices_roi = np.array([[(130, 540), (445, 322), (517, 322), (910, 540)]], dtype=np.int32)

  img_roi = region_of_interest(img_edges, vertices_roi)

  #debug
  #cv2.imwrite("roi.jpg", img_roi)

  return img_roi

**Line Segment Detection (Hough)**

In [11]:
def line_detection(img_roi):

  img_hough, lines = hough_lines(img_roi, 1, np.pi/180, 1, 5, 1)

  #debug
  #cv2.imwrite("hough1_5_1.jpg", img_hough)

  return img_hough, lines

**Line Segment Classification (Left/Right)**

In [12]:
def line_classification(image, lines, img_width):
  left_lines = []
  right_lines = []
  for line in lines:
    for x1,y1,x2,y2 in line:
      if x1 < img_width/2:
        left_lines.append(line)
      else:
        right_lines.append(line)
  return np.array(left_lines), np.array(right_lines)

**Line Fitting (RANSAC)**

In [13]:
#ransac fitting: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RANSACRegressor.html
from sklearn import linear_model

def ransac_line(lines):

  #create ransac linear regression instance
  ransac = linear_model.RANSACRegressor()

  #transpose to get all x's together and y's together
  x1 = lines.T[0]
  y1 = lines.T[1]
  x2 = lines.T[2]
  y2 = lines.T[3]

  #need to reshape, scikit expects X to be 2 dimensional array
  X = np.concatenate((x1[0],x2[0])).reshape(-1,1)

  #y is expected to be 1D, don't need to reshape
  y = np.concatenate((y1[0],y2[0]))

  #ransac algo execution using default variables
  ransac.fit(X,y)

  inlier_mask = ransac.inlier_mask_
  outlier_mask = np.logical_not(inlier_mask)

  slope = ransac.estimator_.coef_
  yint = ransac.estimator_.intercept_

  #debug
  #plt.scatter(X, y)
  #line_y_values = slope * np.array(X) + yint
  #plt.plot(X, line_y_values, color='red', label='Fitted Line')
  #print(inlier_mask)
  #print(outlier_mask)

  return slope, yint


def get_points(lines, img):
  slope, yint = ransac_line(lines)

  #set line to start at bottom of the image (closest to vehicle)
  y1 = img.shape[0]
  x1 = int((y1 - yint)/slope)

  #set line to end 60% height of the image
  y2 = int(y1*0.6)
  x2 = int((y2 - yint)/slope)

  return np.array([x1,y1,x2,y2])

def fit_lines(left_lines, right_lines, image):

  #Run RANSAC
  coordinates_og = np.array([get_points(left_lines, image), get_points(right_lines, image)])

  #fix format to comply with draw_lines
  coordinates_ok = [[[x1, y1, x2, y2]] for x1, y1, x2, y2 in coordinates_og]

  #draw lines
  draw_lines(image, coordinates_ok)

  return image

#debug
#plt.imshow(cv2.cvtColor(debug_img_final, cv2.COLOR_BGR2RGB))
#plt.show()

**Pipeline**

In [17]:
#master function
def detect_lanes(img):

  img_hsv, img_width, img_height = read_image(img)

  img_gray = color_threshold(img_hsv)

  img_blur = smoothing(img_gray)

  img_edges = edge_detector(img_blur)

  img_roi = select_roi(img_edges)

  img_hough, lines = line_detection(img_roi)

  left_lines, right_lines = line_classification(img_hough, lines, img_width)

  #debug: create a test image
  #debug_img_final = img.copy()

  output_image = fit_lines(left_lines, right_lines, img)

  return output_image


#Running the pipeline

img = cv2.imread('solidYellowCurve.jpg')

output_image = detect_lanes(img)

cv2.imwrite("static_output.jpg", output_image)



True

## Test on Videos

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`


In [15]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [16]:
def process_image(image):

    result = detect_lanes(image)
    return result

Let's try the one with the solid white lane on the right first ...

In [ ]:
white_output = 'solidWhiteRight_output.mp4'

clip1 = VideoFileClip("solidWhiteRight.mp4")

white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!

%time white_clip.write_videofile(white_output, audio=False)
#%time clip1.write_videofile(white_output, audio=False)


In [ ]:
yellow_output = 'solidYellowLeft_output.mp4'

clip2 = VideoFileClip('solidYellowLeft.mp4')

yellow_clip = clip2.fl_image(process_image)

%time yellow_clip.write_videofile(yellow_output, audio=False)